Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

### Read Data In

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-07-17 00:54:42--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.213
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  58.9MB/s    in 3.3s    

2020-07-17 00:54:46 (58.9 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [1]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.


Aisles - aisle_id, aisle

Depeartments - department_id, department

Order Products (Prior) - order_id, product_id, add_to_cart_order, reordered

Order Products (Train) - order_id, product_id, add_to_cart_order, reordered

Orders - order_id, product_id, add_to_cart_order, reordered

Products - product_id, product_name, aisle_id, department_id

### Columns needed
products: product_name, product_id
order_products__train/prior : need all except reordered.



In [2]:
import pandas as pd
top_ten = ['Banana', 'Bag of Organic Bananas', 'Organic Strawberries', 
           'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Avocado', 
           'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk']
# filter = products['product_name'].isin(top_ten)



In [3]:
##### YOUR CODE HERE #####
a = pd.read_csv('aisles.csv')      
d = pd.read_csv('departments.csv')
op_p = pd.read_csv('order_products__prior.csv')
op_t = pd.read_csv('order_products__train.csv')
o = pd.read_csv('orders.csv')        
p = pd.read_csv('products.csv')


###Dispaly shapes and column names

#### Display Aisles

In [ ]:
print(a.shape)
print(a.columns)

# I don't need the aisles

(134, 2)
Index(['aisle_id', 'aisle'], dtype='object')


####Display Departments

In [ ]:
print(d.shape)
print(d.columns)

# I don't need departments

(21, 2)
Index(['department_id', 'department'], dtype='object')


####Display Order Products (Prior)

In [ ]:
print(op_p.shape)
print(op_p.columns)

(32434489, 4)
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')


####Display Order Products (Train)

In [ ]:
print(op_t.shape)
print(op_t.columns)

(1384617, 4)
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')


####Display Orders

In [ ]:
print(o.shape)
print(o.columns)
o.head()

# I don't need orders
  # user_id might be useful if I wanted to find individual orders
  # however, we have order_id already to find all the orders to add up

NameError: ignored

####Display Products

In [ ]:
print(p.shape)
print(p.columns)

(49688, 4)
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')


### Merge/Concat data

#### The DataFrames and columns to use

```
order products (train):
  order_id
  product_id
  add_to_cart_ordere
```
```
order products (prior):
  order_id
  product_id
  add_to_cart_order
```
```
products:
  product_id
  product_name
```

#####Concat order_products_xxx

#####Then merge with product

#####Key is product_id

#### Slice the data and preview

In [ ]:
# create a list of the columns to use 
# order_cols_needed = ['order_id', 'product_id', 'add_to_cart_order']
# prod_cols_needed = ['product_id', 'product_name']

In [ ]:
# slice off the useless row
# take a look at the data
op_p = op_p.loc[:,'order_id':'add_to_cart_order']
op_p.head()

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5


In [ ]:
# slice off the useless rows
# take a look at the data
op_t = op_t.loc[:,'order_id':'add_to_cart_order']
op_t.head()

,order_id,product_id,add_to_cart_order
0,1,49302,1
1,1,11109,2
2,1,10246,3
3,1,49683,4
4,1,43633,5


In [ ]:
# slice off useless row
p = p.loc[:,'product_id':'product_name']
p.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [ ]:
p.loc[:,['product_id','product_name']]

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce
...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla"
49684,49685,En Croute Roast Hazelnut Cranberry
49685,49686,Artisan Baguette
49686,49687,Smartblend Healthy Metabolism Dry Cat Food


#### Concatenate the two OP dataframes

In [ ]:
# concat order_products_prior and order_products_train
# ignore the index to keep a continuous index 0 to n-1
op = pd.concat([op_p, op_t], ignore_index=True)
op

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5
...,...,...,...
33819101,3421063,14233,3
33819102,3421063,35548,4
33819103,3421070,35951,1
33819104,3421070,16953,2


In [ ]:
# create new data frame by merging order_products and products
# view data
order_products = pd.merge(op, p, on='product_id')
order_products.sample(3)

,order_id,product_id,add_to_cart_order,product_name
6087593,2273726,28204,24,Organic Fuji Apple
3350797,1691331,27845,3,Organic Whole Milk
26792271,63904,11123,5,Vitamin Water Zero Squeezed Lemonade


In [ ]:
# let's see if we can find some specific data on here
order_products[(order_products['product_id'] == 35123) & (order_products['product_name'] == 'Grilled & Ready Chicken Breast Strips')].sample(3)

,order_id,product_id,add_to_cart_order,product_name
25958630,2597854,35123,2,Grilled & Ready Chicken Breast Strips
25958929,3168631,35123,13,Grilled & Ready Chicken Breast Strips
25958400,2164100,35123,4,Grilled & Ready Chicken Breast Strips


Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [ ]:
# this is a basic answer to the assignment
order_products['product_name'].value_counts().head(10)


Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

#### Reshape Data

#####Table 1 -> Tidy

In [1]:
import pandas as pd
import numpy as np


# creating a simple data frame
table1 = pd.DataFrame(
    [[np.nan, 2],
     [16, 11],
     [3, 1]],
     index=['John Smith', 'Jane Doe', 'Mary Johnson'],
     columns=['treatmenta', 'treatmentb'])
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [3]:
# transpose the table by swapping axes
table2 = table1.copy()
table2

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [4]:
# get the column names
table1.columns.tolist()

['treatmenta', 'treatmentb']

In [5]:
# get the index values
table1.index.tolist()

['John Smith', 'Jane Doe', 'Mary Johnson']

In [6]:
# turn index into a column
table1.reset_index(inplace=True)
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [7]:
# conver the table from 'wide' to 'tidy' using melt()
# id_vars - identified variables
# value_vars - measured variables
tidy = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [8]:
tidy

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [9]:
# rename the columns
tidy = tidy.rename(columns={
    'index':'name',
    'variable':'trt',
    'value':'result'})

# shorten the 'trt' values
# .replace(...) remove treatment, leaving a/b
tidy.trt = tidy.trt.str.replace('treatment', '')
tidy

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


###Table 2 -> Tidy

In [10]:
# table2 -> tidy
table2

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [11]:
# table 2: reset index and melt
tidy2 = table2.reset_index().melt(id_vars='index').rename(columns={ 
    'index':'name',
    'variable':'trt',
    'value':'result'})
tidy2

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [12]:
# rename columns
tidy2['trt'] = tidy2['trt'].str.replace('treatment','')
tidy2 = tidy2.set_index('name')
tidy2

,trt,result
name,,
John Smith,a,NaN
Jane Doe,a,16.0
Mary Johnson,a,3.0
John Smith,b,2.0
Jane Doe,b,11.0
Mary Johnson,b,1.0


In [ ]:
# clean up the column names and set index
tidy2['trt'] = tidy2['trt'].str.replace('treatment', '')
tidy2 = tidy2.set_index('name')
tidy2

,trt,result
name,,
John Smith,a,NaN
Jane Doe,a,16.0
Mary Johnson,a,3.0
John Smith,b,2.0
Jane Doe,b,11.0
Mary Johnson,b,1.0


#####Tidy -> Table 1

In [ ]:
wide = tidy.pivot_table(index='name', columns='trt', values='result')
wide = wide.rename(columns={
    'trt':'',
    'a':'treatmenta',
    'b':'treatmentb'})
# can't seem to remove 'trt
# tried wide.index.name = ''
wide

trt,treatmenta,treatmentb
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


#####Tidy 2 -> Table 2

In [ ]:
# tidy -> table2
tidy2.pivot_table(index='name', columns='trt', values='result').T

name,Jane Doe,John Smith,Mary Johnson
trt,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


#### Tidy2 -> Table 2

In [13]:
wide = tidy2.pivot_table(index='name', columns='trt', values='result')
wide

trt,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


#####Flights

Load seaborn's flights dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [15]:
import seaborn as sns
flights = sns.load_dataset('flights')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [17]:
# Flights Pivot Table
flights.pivot_table(index='year', columns='month')

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [4]:
##### YOUR CODE HERE #####
# a d op_p op_t o p
print(a.columns)        # dont need
print(d.columns)        # dont need
print(op_p.columns)     # order_id, product_id
print(op_t.columns)     # order_id, product_id
print(o.columns)        # order_id, order_hour_of_day
print(p.columns)        # product_id, product_name

Index(['aisle_id', 'aisle'], dtype='object')
Index(['department_id', 'department'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')


#####Slice columns as needed

In [5]:
op_p = op_p.loc[:,'order_id':'product_id']
op_p.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [6]:
# splice dataframes as needed
op_t = op_t.loc[:,'order_id':'product_id']
op_t.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [7]:
p = p.loc[:,'product_id':'product_name']
p.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [9]:
# we only want late orders, is 1600hrs late enough?
o = o.loc[:, ['order_id', 'order_hour_of_day']]
o = o[o['order_hour_of_day'] > 16]
o.head()

,order_id,order_hour_of_day
27,444309,19
29,2037211,18
30,2710558,17
33,3225766,17
36,521107,18


#####Concat order products train/prior

In [10]:
# concat order_products_prior and order_products_train
# ignore the index to keep a continuous index 0 to n-1
op = pd.concat([op_p, op_t], ignore_index=True)
op

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035
...,...,...
33819101,3421063,14233
33819102,3421063,35548
33819103,3421070,35951
33819104,3421070,16953


#####Merge 

In [11]:
# create new data frame by merging order_products and products
# view data
order_products_stretch = pd.merge(op, p, on='product_id')
order_products_stretch.sample(3)

,order_id,product_id,product_name
8533303,3284096,9387,Granny Smith Apples
26836409,2816486,31394,100 Calorie Pretzels Packs
20256631,1033587,17789,Soy Free Buttery Spread


In [12]:
# create new data frame by merging order_products and objects
# view data
order_products_stretch = pd.merge(order_products_stretch, o, on='order_id')
order_products_stretch.sample(3)

,order_id,product_id,product_name,order_hour_of_day
1223153,1646543,47672,Uncured Hickory Smoked Sunday Bacon,21
4613885,2165334,24852,Banana,18
6913410,317928,27316,Trop50 No Pulp Orange Juice w/ Calcium + Vitam...,18


In [20]:
order_products_stretch = order_products_stretch[order_products_stretch['order_hour_of_day'] >= 18]

# df['name'].value_counts()[df['name'].value_counts() == df['name'].value_counts().max()]


order_products_stretch['product_name'].value_counts() > 2900

Banana                                                True
Bag of Organic Bananas                                True
Organic Strawberries                                  True
Organic Baby Spinach                                  True
Organic Hass Avocado                                  True
                                                     ...  
Peanut Butter Panic Ice Cream                        False
Apis Mellifica 30 C                                  False
Passionfruit Papaya Tea Bags                         False
Jalapeño Pumpkin Seeds                               False
New Orleans Style Garlic Butter Flavored Rice Mix    False
Name: product_name, Length: 46781, dtype: bool

In [26]:
order_products_stretch[order_products_stretch['order_hour_of_day'] >= 16].sample(5)

,order_id,product_id,product_name,order_hour_of_day
6000842,2582587,47900,"Uncured, No Sugar, Hickory Smoked Bacon",16
20017892,2764966,47532,Instant Whole Grain Brown Rice,16
25656932,50147,14126,Gluten and Dairy Free Rice Macaroni and Cheeze,17
10604466,2118611,41909,Bread & Butter Pickle Chips,22
28267241,1393689,17993,Shredded Colby & Monterey Jack Cheeses,17


In [21]:
op_filtered = order_products_stretch[(order_products_stretch['order_hour_of_day'] > 18)]
op_filtered.head(50)

,order_id,product_id,product_name,order_hour_of_day
0,582,33120,Organic Egg Whites,19
1,582,28985,Michigan Organic Kale,19
2,582,33754,Total 2% with Strawberry Lowfat Greek Strained...,19
3,582,21903,Organic Baby Spinach,19
4,582,25890,Boneless Skinless Chicken Breasts,19
5,582,26128,Organic Mango Chunks,19
6,582,47626,Large Lemon,19
7,582,39275,Organic Blueberries,19
8,582,17872,Total 2% Lowfat Plain Greek Yogurt,19
9,582,9076,Blueberries,19


In [22]:
op_filtered['product_name'].value_counts().head(25)

Banana                      61711
Bag of Organic Bananas      50602
Organic Strawberries        37462
Organic Baby Spinach        32727
Organic Hass Avocado        28343
Organic Avocado             23200
Organic Whole Milk          19807
Organic Raspberries         19079
Strawberries                18998
Large Lemon                 18820
Limes                       17474
Organic Blueberries         14747
Organic Zucchini            14275
Organic Yellow Onion        13913
Organic Garlic              13668
Cucumber Kirby              13080
Organic Grape Tomatoes      11826
Organic Lemon               11489
Seedless Red Grapes         11401
Organic Cucumber            11105
Organic Fuji Apple          11020
Apple Honeycrisp Organic    10776
Organic Baby Carrots        10616
Honeycrisp Apple            10422
Carrots                     10190
Name: product_name, dtype: int64

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [ ]:
##### YOUR CODE HERE #####